In [1]:
import pandas as pd
from main import Platform, Coin

import defi_tools as dft

portfolio = pd.read_csv('portfolio.csv')
portfolio.fillna(0, inplace=True)
portfolio.drop('Unnamed: 0', axis=1, inplace=True)
portfolio.rename(columns={'Unnamed: 1':'id'}, inplace=True)
portfolio.rename(columns={'Coinbase Pro':'Coinbase_Pro'}, inplace=True)
portfolio.set_index('id', inplace=True)


In [2]:
portfolio.head()

,Ledger,Coinbase,Coinbase_Pro,BlockFi,TerraStation,KuCoin,Trust,Metamask,CDC,Anchor
id,,,,,,,,,,
ethereum,1.01,0.000000,0.35,0.849155,0.0,0.000000,0.0,0.0315,0.0495,0.000
terrausd,0.00,0.000000,0.00,0.000000,7004.0,0.000000,0.0,0.0000,0.0000,480.234
usd-coin,0.00,0.003664,1.00,7502.660000,0.0,0.000000,0.0,0.0000,0.0000,0.000
bitcoin,0.00,0.001958,0.07,0.034949,0.0,0.008771,0.0,0.0000,0.0040,0.000
cardano,0.00,0.000000,2755.00,0.000000,0.0,0.195800,0.0,0.0000,0.0000,0.000


In [3]:
platforms = {k:'' for k in portfolio.columns}
ids = portfolio.index.tolist()
prices = {i:dft.geckoPrice(i, 'usd')[i]['usd'] for i in ids} 

In [4]:
for p in platforms.keys():
    platforms[p] = Platform(p)
    for i in ids:
        coin = Coin(coin=i, price=float(prices[i]), holdings=portfolio.loc[i][p])
        platforms[p].add(coin)

In [5]:
platforms['Coinbase_Pro'].value()

13350.47731554

In [6]:
platforms['Coinbase_Pro'].tokens(token='ethereum')

0.35